# Webcam Real-Time Object Detection (Blackjack Counter)

## 1. Import Dependencies

In [1]:
import os
import time
import tensorflow as tf
import cv2
import numpy as np

#from utils import label_map_util
from PIL import Image
#from google.colab.patches import cv2_imshow
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from IPython.display import HTML
from base64 import b64encode


# 1.1 Filepaths

In [2]:
PRETRAINED_MODEL_NAME='ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'

In [3]:
model_filepath= os.path.join('trained_models',f'{PRETRAINED_MODEL_NAME}')
pipeline_filepath = os.path.join('pipeline_configs',f'{PRETRAINED_MODEL_NAME}_pipeline.config')

In [4]:
from pathlib import Path

In [5]:
os.path.join(os.getcwd(),'LABEL_MAPS','labelmap.pbtxt') 

'C:\\Programming\\Blackjack-Card-Counter-Object-Detection-main\\2.Real_Time_Object_Detection\\LABEL_MAPS\\labelmap.pbtxt'

In [6]:
os.path.join(os.getcwd(),'LABEL_MAPS','labelmap.pbtxt') 

'C:\\Programming\\Blackjack-Card-Counter-Object-Detection-main\\2.Real_Time_Object_Detection\\LABEL_MAPS\\labelmap.pbtxt'

In [7]:
previous_dir = os.getcwd()[:-29] #i know that this is lazy, but I was rushing

In [8]:
#!copy C:\\Programming\\Blackjack-Card-Counter-Object-Detection-main\\1.Train_and_Export_Neural_Network\\trained_models\\ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8\\inference_graph\\saved_model\\x.txt"

## 2. Import Saved Model and Label Map

In [9]:
labelmap_dir = os.path.join(previous_dir,"1.Train_and_Export_Neural_Network\\LABEL_MAPS\\labelmap.pbtxt")
category_index=label_map_util.create_category_index_from_labelmap(labelmap_dir,use_display_name=False)

#category_index=label_map_util.create_category_index_from_labelmap("labelmap.pbtxt",use_display_name=False)




#category_index=label_map_util.create_category_index_from_labelmap("C:/Programming/BJC_Notebook/LABEL_MAPS/labelmap.pbtxt",use_display_name=False)

### A. SSD_RESNET152_V1_FPN_640x640_COCO17_TPU8

In [10]:
#original ssd_resnet152
#PATH_TO_SAVED_MODEL = "C:/Programming/BJC_Notebook/trained_models/SSD_RESNET152_V1_FPN_640x640_COCO17_TPU8/inference_graph/saved_model"

#PATH_TO_SAVED_MODEL = os.path.join(os.getcwd()[:-29],"1.Train_and_Export_Neural_Network\\{}\\inference_graph\\saved_model".format(model_filepath))
PATH_TO_SAVED_MODEL = os.path.join(previous_dir,"1.Train_and_Export_Neural_Network\\{}\\inference_graph\\saved_model".format(model_filepath))


### B. SSD_MOBILENET_V2_FPNLITE_640x640_COCO17_TPU-8

In [11]:
#PATH_TO_SAVED_MODEL = "C:/Programming/BJC_Notebook/trained_model/inference_graph/saved_model"

### C. EFFICIENTDET_D1_COCO17_TPU-32

In [12]:
#PATH_TO_SAVED_MODEL = "C:/Programming/BJC_Notebook/trained_models/efficientdet_d1_coco17_tpu-32/inference_graph/saved_model"

### D. Faster_R-CNN_ResNet101_V1_640x640

In [13]:
#"C:/Programming/Bjc_notebook/trained_models/faster_rcnn_model_directory/inference_graph/saved_model"

# 3. Draw Bounding Boxes and Save New Detections

In [14]:
def bbox_and_count(image, bboxes, labels, scores, thresh):
    (h, w, d) = image.shape
    for bbox, label, score in zip(bboxes, labels, scores):
        #print(bbox)
        if score > thresh:
            #add this detection to current_detections for total count if it hasn't been detected before.
            if label not in current_detections:
                current_detections.append(label)
            xmin, ymin = int(bbox[1]*w), int(bbox[0]*h)
            xmax, ymax = int(bbox[3]*w), int(bbox[2]*h)
            

            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255,153,255), 2)
            cv2.putText(image, f"{label}: {int(score*100)} %", (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    return image

# 4. Differentiate between Player and Dealer Hand

In [15]:
#This function determines if the detected card belongs to the player or the dealer based on it's location
#It will then call hand_total() to calculate the corresponding player/dealer's hand total
#It will also call manage_user_hand to take care of mis-detections
def player_or_dealer_hand(bboxes,labels,scores,score_thresh, dealer_detections, player_detections,dealer_hand_total,player_hand_total):
    #dealer
    soft_total = False

    for bbox, label, score in zip(bboxes, labels, scores):
        if score > score_thresh:
            if (bbox[0]<=0.5 and bbox[2]<=0.5):
                #add or remove a detection
                dealer_detections = manage_user_hand(bbox,label, dealer_detections)

                #calculate the sum of the dealer's hand
                dealer_hand_total = hand_total(dealer_detections[0])[0]


            #player
            else:
                #add or remove a detection
                player_detections = manage_user_hand(bbox,label, player_detections)

                #calculate the sum of the player's hand
                player_hand_total = hand_total(player_detections[0])[0]

    return dealer_detections, dealer_hand_total, player_detections, player_hand_total

In [16]:
#This function handles mis-detections; i.e. remove card detections from previous round
#detections is a 2-D array that stores: [[detected card],[mis-detection counter]]
#every time a card isn't detected, we will increase the counter for misdetection counter and once the
#card has not been detected for >20 times, that detection will be popped from the user's hand

def manage_user_hand(bbox,label,detections):
    if label not in detections[0]:
        detections[0].append(label)
        detections[1].append(0)

    for index,x in enumerate(detections[0]):
        if x != label:
            detections[1][index] += 1
        else: 
            detections[1][index] = 0 
        if detections[1][index]>10:
            detections[0].remove(x)
            detections[1].pop(index)
            
            #hand_size = 5 #reset to recalculate
    
    return detections

# 5. Card Counting Functions

## 5.0 CURRENT COUNT OF DECK

In [17]:
#This function keeps track of the ENTIRE deck's count based on detected cards.
#The key flaw for this method is that misdetections will skew the deck's count until the "misdetected" card comes out,
#this error can be reduced by increasing the neural network's threshold for detecting a card
#if card detected is 2<->6, add 1 to count, if 10,J,Q,K,A, Subtract 1 

#more efficient way is to just pass count as a parameter and add/subtract based on new detection, however there are only 52 cards in a deck. Performance is not that big of a difference
def current_count(current_detections):
    count=0
    for i in current_detections:
            x=str(i)[:-1]
            if x.isalpha() or (int(x)==10):
                count -=1
            elif (int(x)<7):
                count= count + 1
    return count, len(current_detections)



## 5.1.0 Calculate Total Sum of a Hand 

### 5.1.1 Calculate Sum of Cards excluding Aces First

In [18]:
#calculate sum of all non-ace cards in hand first

def calculate_hand(hand_total,x):
    if x.isalpha(): #J,Q,K 
        hand_total+=10
    else: #2,3,4,5,6,7,8,9,10
        hand_total=hand_total+int(x)
    return hand_total

### 5.1.2 Calculate Sum of Cards Including Aces

In [19]:
#this function handles calculating the soft/hard totals of a hand given a list of detections
#example : A,6 = 7/17, A,A,8 = 10/20, A,10,2 = 13

#Calculate the hand's soft/hard total if it contains aces

def hand_total(input_hand):
    detections_list=input_hand
    ace_count = 0 
    card_total = 0
    soft_total = False
    for i in detections_list:
        i = str(i)[:-1] #trim the suit character from each string in the list
        
        if (i =='A'): #count number of aces in hand
            ace_count+=1
        else: #only add the non-aces to the card total first
            card_total = calculate_hand(card_total,i)

    #adding aces to card total IF they exist:
    remainder = 21-card_total
    ace_11 = 0
    
    # >1 ace in hand
    if (ace_count>0 and remainder/11>1):
        ace_11 = 1
        ace_count -= 1 #subtract the ace that became '11'
 
        #if having an ace == 11 causes a bust, then every ace must equal 1
    
        if ((card_total + ace_11*11 + (ace_count))>=21): 
            #convert the ace_11 to == 1 
            ace_11 -=1
            ace_count +=1
        
        card_total = card_total + ace_count * 1 + ace_11 * 11
    else: 
        card_total = card_total + ace_count * 1
    
    if ace_11>0:
        soft_total=True
    
    return card_total, soft_total

# 6.0 Calculate Best Move based on Dealer's hand, Player's Hand, and Current Count

In [20]:
from S17_Calculate_Best_Move import *

In [21]:
#run this to see what the functions will return 

dealer_detections = [['2S','3S','4S','5S','6S','7S','8S','9S','10S','AS'],[5]]
player_detections = [['6C','2S'],[5,9]]
dealer_hand_total_LIST = [2,3,4,5,6,7,8,9,10,11] 
dealer_hand_total = 10
count= 3

for i in dealer_hand_total_LIST:
    #print("Ace pair:")
    print("Player Hand: ",player_detections[0])
    print("Dealer total: ",i,"\n\nPlay = ",(soft_17(player_detections, i, count)))
    print()


best_move = soft_17(player_detections,dealer_hand_total, count

# 7.0 Initialize Variables before Main Loop

In [22]:
#Used for total deck count

current_detections= [] #Stores all detected cards
count=0
length = 0

#Used for player/dealer hand totals

dealer_hand_total=0
player_hand_total=0

#empty list for cards and # of loop iterations that they have not been detected while in hand
        #[['QS'],[10]] ==> QS has been in the hand, and has not been detected for 10 iterations, if this counter hits 20, remove QS from detection list
dealer_detections=[[],[]]
player_detections=[[],[]]

best_move = 'Hit'

toggle_mode = False #False = no overlay, True = player vs dealer

# define a video capture object
video_capture = cv2.VideoCapture(0)

# Load the model
print("Loading saved model ...")
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
print("Model Loaded!")
start_time = time.time()

#Make sure to feed images the same resoltuion as the neural network was trained on.
def make_640p():
    video_capture.set(3, 640)
    video_capture.set(4, 640)
make_640p()

frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

fps = int(video_capture.get(cv2.CAP_PROP_FPS))
size = (frame_width, frame_height)

#variables used for text/shape coordinates
half_height=int(frame_height/2)


dealer_hand_text_coords=tuple([int(0.1875*frame_width),int(0.052083*frame_height)])
player_hand_text_coords=tuple([int(0.15625*frame_width),int(0.90625*frame_height)])
    
dealer_total_text_coords=tuple([int(0.1875*frame_width),int(0.104167*frame_height)])
player_total_text_coords=tuple([int(0.15625*frame_width),int(0.95833*frame_height)])
    
deck_count_text_coords=tuple([int(0.039063*frame_width),int(0.104167*frame_height)])
fps_text_coords=tuple([int(0.039063*frame_width),int(0.052083*frame_height)])

best_move_text_coords=tuple([int(0.7*frame_width),int(0.95*frame_height)])


Loading saved model ...
Model Loaded!


# 8. Main Loop

In [23]:

print("Press 'q' to end the program.")
print("Press 'w' to toggle between Card Counting and Player vs. Dealer modes.")
print("Press 'r' to reset the count and the player/dealer hands and totals.")


while(True):
      
    # Capture the video frame by frame
    ret, frame = video_capture.read()

    #toggle player vs dealer or basic card count mode
    if cv2.waitKey(1) & 0xFF == ord('w'):
        toggle_mode = not(toggle_mode)
        
    
    # Kill switch
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    #Reset memory switch
    
    if cv2.waitKey(1) & 0xFF == ord('r'): 
        
        current_detections=[]
        count=0
        length=0
        
        dealer_detections=[[],[]]
        player_detections=[[],[]]
        
        dealer_hand_total=0
        player_hand_total=0
        


    frame = cv2.flip(frame, 1)
    image_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
      # The model expects a batch of images, so also add an axis with `tf.newaxis`.
    input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]

      # Pass frame through detector
    detections = detect_fn(input_tensor)
    
    #print(detections)
    #detections == "raw_detection_boxes", "detection_multiclass_Scores", "detection_anchor_+indices", "raw_detection_scores", 'detection_scores', 'detection_boxes',
        
    # Set detection parameters

    score_thresh = 0.75   # Minimum threshold for object detection
    max_detections = 20

      # All outputs are batches tensors.
      # Convert to numpy arrays, and take index [0] to remove the batch dimension.
      # We're only interested in the first num_detections.
    scores = detections['detection_scores'][0, :max_detections].numpy()
    bboxes = detections['detection_boxes'][0, :max_detections].numpy()
    labels = detections['detection_classes'][0, :max_detections].numpy().astype(np.int64)
    labels = [category_index[n]['name'] for n in labels]

    # Display detections
    
    #1. DRAW BBOXES AND COUNT DECK
    bbox_and_count(frame, bboxes, labels, scores, score_thresh)
    
    
    #2. Check if a new card has been detected. If it has, then re-calculate the current count of the deck
    if len(current_detections)>length:
        count,length = current_count(current_detections)
        
    #3. CALCULATE PLAYER AND DEALER HAND TOTALS based on new detection location and confidence
    dealer_detections,dealer_hand_total,player_detections, player_hand_total =player_or_dealer_hand(bboxes,labels,scores,score_thresh, dealer_detections, player_detections,dealer_hand_total,player_hand_total)
   
    
    
    #Current dealer and player hands
    
    if toggle_mode == True:
    
        cv2.putText(frame, f"Dealer hand = : {dealer_detections}", dealer_hand_text_coords, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,255), 1)  
        cv2.putText(frame, f"Player hand = : {player_detections}", player_hand_text_coords, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,255), 1)

        #dealer and player hand totals

        cv2.putText(frame, f"DEALER TOTAL = : {dealer_hand_total}", dealer_total_text_coords, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,255), 1)
        cv2.putText(frame, f"Player TOTAL = : {player_hand_total}", player_total_text_coords, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,255), 1)

        #split screen in half to show dealer vs player
        cv2.line(frame,(0,half_height),(frame_width,half_height),(255,0,255),2)
        
        #Calculate best move for player based on player's hand, dealer's hand total, and current count
        
        best_move = soft_17(player_detections,dealer_hand_total, count)
        cv2.putText(frame, f"Best Move: {best_move}", best_move_text_coords, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,255), 1)

        
    cv2.putText(frame, f"Count: {count}", deck_count_text_coords, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,255), 1)
    
    #fps calculation
    end_time = time.time()
    fps = int(1/(end_time - start_time))
    start_time = end_time
    cv2.putText(frame, f"FPS: {fps}", fps_text_coords, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,255), 1)
    cv2.imshow("Object Detector", frame)
      
    #Write output video
    #result.write(frame)

# After the loop release the cap object
video_capture.release()
# Destroy all the windows
cv2.destroyAllWindows()


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
dealer_hand_total